# C++ 部署

C++ 部署分为两种方式：

1. 方式 A：直接利用 TVM 源码库构建的文件夹 `build/`；
2. 方式 B：自定义 TVM 运行时打包脚本。

## 方式 A

为了便捷，在 TVM 源码库的 apps 下载创建目录 `deploy_cpp`。

目录组织如下：

![C++ 部署方式 A](images/deploy_cpp.png)

```{note}
- `python.ipynb` 用于编写输出动态库到 `lib/`。
- `src/` 用于编写 C++ 用例。
- `include/` 存储一些头文件。
- `outputs/` 存储一些结果文件，比如 C++ 编译结果。
- `run.sh` 定义需要处理的任务。
- `Makefile` 提供 C++ 编译。
```

下面提供一个简单测试样例：

````{tab-set-code}

```{literalinclude} ./deploy_cpp/Makefile
:language: Makefile
```

```{literalinclude} ./deploy_cpp/run.sh
:language: bash
```

```{literalinclude} ./deploy_cpp/src/test_alloc_array.cc
:language: c++
```
````

## 方式 B

同样提供简单测试样例（对方式 A 修改如下）：


````{tab-set-code}

```{literalinclude} ./deploy_cpp_pack/Makefile
:language: Makefile
```

```{literalinclude} ./deploy_cpp_pack/src/libtvm_runtime_pack.cc
:language: c++
```
````

## Relay

也可以加载 Python 端导出的动态库：

```c++
/*!
 * \brief 加载并运行 TVM module.s 的示例代码
 * \file deploy_graph.cc
 */
#include <dlpack/dlpack.h>
#include <tvm/runtime/module.h>
#include <tvm/runtime/packed_func.h>
#include <tvm/runtime/registry.h>

tvm::runtime::NDArray graph_executor_output(std::string fname, DLDevice dev) {
  LOG(INFO) << "运行 graph executor";
  
  // 在库中加载
  tvm::runtime::Module mod_factory = tvm::runtime::Module::LoadFromFile(fname);
  // 创建 graph executor 模块
  tvm::runtime::Module gmod = mod_factory.GetFunction("default")(dev);
  tvm::runtime::PackedFunc set_input = gmod.GetFunction("set_input");
  tvm::runtime::PackedFunc get_output = gmod.GetFunction("get_output");
  tvm::runtime::PackedFunc run = gmod.GetFunction("run");

  // 使用 C++ API
  tvm::runtime::NDArray x = tvm::runtime::NDArray::Empty({2, 2}, DLDataType{kDLFloat, 32, 1}, dev);

  for (int i = 0; i < 2; ++i) {
    for (int j = 0; j < 2; ++j) {
      static_cast<float*>(x->data)[i * 2 + j] = i * 2 + j;
    }
  }

  // 设置正确的 input
  set_input("x", x);
  
  // 运行代码
  run();
  // 获取 output
  return get_output(0);
}

int main() 
{
  DLDevice dev{kDLCPU, 0};
  tvm::runtime::NDArray y = graph_executor_output("lib/test_relay_add.so", dev);
  for (int i = 0; i < 2; ++i) {
    for (int j = 0; j < 2; ++j) {
      ICHECK_EQ(static_cast<float*>(y->data)[i * 2 + j], i * 2 + j + 1);
    }
  }
  return 0;
}
```